In [11]:
import numpy as np
from scipy.optimize import minimize

# ===================== 常量参数：请按实际工艺 / 设计填 =====================
beta_n = 3.9e-3     # nMOS β_n
beta_p = 1.38e-3    # pMOS β_p
lam = 0.05          # 渠长调制 λ
Vth = 0.5           # 阈值电压
A_req = 500.0       # 增益要求
R_L = 1e4           # 负载电阻

# W_i, L_i 如果在这个问题里视为常数，请在这里给出：
# 你可以先随便设一组满足工艺边界的 W, L，用来求 s_i=W_i/L_i
# 这里先举例：全部用某个默认尺寸（你自己改！）
W = np.array([1e-6]*9)  # 占位：9 个晶体管的宽度
L = np.array([1e-6]*9)  # 占位：9 个晶体管的长度

W_min = np.array([12e-9]*9)
W_max = np.array([1e-2]*9)
L_min = np.array([15e-9]*9)
L_max = np.array([1e-2]*9)

# 检查一下常量是否满足边界（调参时自己确认）
assert np.all(W >= W_min) and np.all(W <= W_max)
assert np.all(L >= L_min) and np.all(L <= L_max)

# ===================== 变量拆包 / 装包 =====================

def unpack(x):
    """
    x: 长度 37
    I[0..8], Vov[0..8], dV[0..8], s[0..8], Vdd
    """
    I = x[0:9]
    Vov = x[9:18]
    dV = x[18:27]
    s = x[27:36]
    Vdd = x[36]
    return I, Vov, dV, s, Vdd

# ===================== 目标函数 =====================

def objective(x):
    I, Vov, dV, s, Vdd = unpack(x)
    # 目标: sum_{i in {5,6,8}} I_i * Vdd = I_5 + I_6 + I_8 (注意 0-based index)
    return Vdd * (I[4] + I[5] + I[7])

# ===================== 约束函数 =====================

def constraints_list():
    cons = []

    # 1) W_i / (L_i s_i) = 1   ->  W_i/(L_i*s_i) - 1 = 0
    def make_s_eq(i):
        def fun(x):
            I, Vov, dV, s, Vdd = unpack(x)
            return W[i] / (L[i] * s[i]) - 1.0
        return {'type': 'eq', 'fun': fun}
    for i in range(9):
        cons.append(make_s_eq(i))

    # 2) 器件方程 I_i = 0.5 * β * s_i * Vov_i^2 * (1 + λ(Vov_i + ΔV_i))
    nmos_idx = [0, 1, 4, 5, 6]  # {1,2,5,6,7} -> 0-based
    pmos_idx = [2, 3, 7, 8]     # {3,4,8,9}
    def make_device_eq(i, beta):
        def fun(x):
            I, Vov, dV, s, Vdd = unpack(x)
            rhs = 0.5 * beta * s[i] * (Vov[i]**2) * (1.0 + lam*(Vov[i] + dV[i]))
            return I[i] - rhs
        return {'type': 'eq', 'fun': fun}
    for i in nmos_idx:
        cons.append(make_device_eq(i, beta_n))
    for i in pmos_idx:
        cons.append(make_device_eq(i, beta_p))

    # 3) (I1 + I2) / I6 = 1
    def fun_I12_over_I6(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return (I[0] + I[1]) / I[5] - 1.0
    cons.append({'type': 'eq', 'fun': fun_I12_over_I6})

    # 4) I_i / I_j = 1 for specified current pairs
    equal_pairs = [(8, 4), (0, 2), (1, 3), (6, 7)]
    def make_current_ratio_eq(ii, jj):
        def fun(x):
            I, Vov, dV, s, Vdd = unpack(x)
            return I[ii] / I[jj] - 1.0
        return {'type': 'eq', 'fun': fun}
    for ii, jj in equal_pairs:
        cons.append(make_current_ratio_eq(ii, jj))

    # 5) ΔV_i = V_th, i in {3,5,9} -> indices 2,4,8
    fixed_dV_idx = [2, 4, 8]
    for i in fixed_dV_idx:
        def make_dv_eq(ii):
            def fun(x):
                I, Vov, dV, s, Vdd = unpack(x)
                return dV[ii] - Vth
            return {'type': 'eq', 'fun': fun}
        cons.append(make_dv_eq(i))

    # 6) Vov5 = Vov6 = Vov7  -> indices 4,5,6
    def fun_vov_5_6(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return Vov[4] - Vov[5]
    def fun_vov_6_7(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return Vov[5] - Vov[6]
    cons.append({'type': 'eq', 'fun': fun_vov_5_6})
    cons.append({'type': 'eq', 'fun': fun_vov_6_7})

    #    Vov1 = Vov2  -> indices 0,1
    def fun_vov_1_2(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return Vov[0] - Vov[1]
    cons.append({'type': 'eq', 'fun': fun_vov_1_2})

    #    Vov3 = Vov4  -> indices 2,3
    def fun_vov_3_4(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return Vov[2] - Vov[3]
    cons.append({'type': 'eq', 'fun': fun_vov_3_4})

    # 7) 几个含 Vdd 的电压关系（全部写成 eq）
    # 2(Vov2+Vov4+ΔV2+ΔV4)/Vdd = 1
    def fun_eq1(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return 2*(Vov[1] + Vov[3] + dV[1] + dV[3]) / Vdd - 1.0
    cons.append({'type': 'eq', 'fun': fun_eq1})

    # (Vov7+Vov8+ΔV7+ΔV8)/Vdd = 1
    def fun_eq2(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return (Vov[6] + Vov[7] + dV[6] + dV[7]) / Vdd - 1.0
    cons.append({'type': 'eq', 'fun': fun_eq2})

    # 2(Vov1+Vov6+ΔV6+Vth)/Vdd = 1
    def fun_eq3(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return 2*(Vov[0] + Vov[5] + dV[5] + Vth) / Vdd - 1.0
    cons.append({'type': 'eq', 'fun': fun_eq3})

    # 2(ΔV2+Vov8)/Vdd = 1
    def fun_eq4(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return 2*(dV[1] + Vov[7]) / Vdd - 1.0
    cons.append({'type': 'eq', 'fun': fun_eq4})

    # 2(ΔV1+Vov3)/Vdd = 1
    def fun_eq5(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return 2*(dV[0] + Vov[2]) / Vdd - 1.0
    cons.append({'type': 'eq', 'fun': fun_eq5})

    # 8) Vdd/12 <= 1  ->  1 - Vdd/12 >= 0
    def fun_vdd_ineq(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return 1.0 - Vdd/12.0
    cons.append({'type': 'ineq', 'fun': fun_vdd_ineq})

    # 9) 增益约束:
    # A_req(λI2+λI4)(λI7+λI8+1/RL)/(β_n β_p s1 s8 Vov1 Vov8) <= 1
    # -> 1 - 上式 >= 0
    def fun_gain_ineq(x):
        I, Vov, dV, s, Vdd = unpack(x)
        num = A_req * (lam*I[1] + lam*I[3]) * (lam*I[6] + lam*I[7] + 1.0/R_L)
        den = beta_n * beta_p * s[0] * s[7] * Vov[0] * Vov[7]
        val = num / den
        return 1.0 - val
    cons.append({'type': 'ineq', 'fun': fun_gain_ineq})

    return cons

# ===================== 初始值 & 边界 =====================

# 简单初始化：全部设成正的常数
# 你可以多试几组不同的初始值（非常重要！）
I0 = np.ones(9) * 1e-5
Vov0 = np.ones(9) * 0.1
dV0 = np.ones(9) * Vth
s0 = W / L          # 初始 s 直接设成 W/L
Vdd0 = 1.2

x0 = np.concatenate([I0, Vov0, dV0, s0, np.array([Vdd0])])

# 每个变量的下界设成一个小正数，避免除零
eps = 1e-6
bounds = [(eps, None)] * len(x0)

# ===================== 求解 =====================

cons = constraints_list()

res = minimize(
    objective,
    x0,
    method='SLSQP',
    bounds=bounds,
    constraints=cons,
    options={'maxiter': 5000, 'ftol': 1e-9, 'disp': True}
)

print("status:", res.status, res.message)
print("optimal objective (power):", res.fun)
I_opt, Vov_opt, dV_opt, s_opt, Vdd_opt = unpack(res.x)
print("Vdd_opt =", Vdd_opt)
print("I_opt =", I_opt)
print("Vov_opt =", Vov_opt)
print("dV_opt =", dV_opt)
print("s_opt =", s_opt)

# 计算增益约束中的 val 以便检查
gain_val = (
    (beta_n * beta_p * s_opt[0] * s_opt[7] * Vov_opt[0] * Vov_opt[7]) / (lam * I_opt[1] + lam * I_opt[3]) / (lam * I_opt[6] + lam * I_opt[7] + 1.0 / R_L)
    
)
print("gain constraint val =", gain_val)


Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 7.211894185850803e-05
            Iterations: 19
            Function evaluations: 580
            Gradient evaluations: 15
status: 8 Positive directional derivative for linesearch
optimal objective (power): 7.211894185850803e-05
Vdd_opt = 12.000000000005429
I_opt = [1.01139011e-06 1.00924983e-06 1.00214078e-06 1.00000000e-06
 1.39596209e-06 1.97473681e-06 2.64739260e-06 2.63921292e-06
 1.40565286e-06]
Vov_opt = [0.02228968 0.02228968 0.04213547 0.04213547 0.02947202 0.02947202
 0.02947202 0.06815118 0.05339232]
dV_opt = [5.95786462e+00 5.93184880e+00 5.00000000e-01 4.10781800e-01
 5.00000000e-01 5.85529429e+00 1.19023757e+01 1.00000000e-06
 5.00000000e-01]
s_opt = [1. 1. 1. 1. 1. 1. 1. 1. 1.]
gain constraint val = 811.6533210396357
